In [1]:
import os
import json
et_dir="autograded/user001/entrance_test"
cells_all=[]
for file in os.listdir(et_dir):
    if file.endswith(".ipynb"):
        with open(os.path.join(et_dir, file), mode= "r", encoding= "utf-8") as f:
            myfile = json.loads(f.read())
            cells_all.extend(myfile['cells'])

In [3]:
tags_standard = ['visualization', 'functionals','sequences', 'conditionals', 'loops', 'operators', 'data']
counts=[0,0,0,0,0,0,0]
totals = [0,0,0,0,0,0,0]
for cell in cells_all:
    if 'tags' in cell['metadata']:
        tags = cell['metadata']['tags']
        for tag in tags:
            for index in range(len(tags_standard)):
                if tag == tags_standard[index]:
                    totals[index]= totals[index]+1
                    if len(cell['outputs']) == 0:
                        counts[index]= counts[index]+1
                    break

print(counts)
print(totals)
                
        
        

[0, 0, 0, 0, 0, 1, 10]
[18, 15, 19, 13, 20, 5, 14]


In [5]:
from bokeh.io import output_notebook, show
from bokeh.models import HoverTool, Paragraph, Div, ColumnDataSource
from bokeh.plotting import figure

output_notebook()

concepts = ['Visualization', 'Functionals', 'Sequences', 'Conditionals', 'Loops', 'Operators', 'Data']
counts = [1, 2, 4, 8, 7, 9, 10]
totals = [10,10,10,10,10,10,10]
max_value = max(totals)
description = ['Visualization involves viewing your data using plots',
               'Functionals are blocks that your program is divided into',
               'Sequences are  a particular activity or task is expressed as a series of individual steps or instructions that can be executed by the computer',
               'Loops allow you to repeat a statement multiple times',
               'Conditionals allow you to make logical decisions based on a condition',
               'Operators provide support for mathematical, logical, and string expressions',
               'Data involves storing, retrieving, and updating values']


colors=[]
for i in range(len(counts)): 
    if counts[i] < (0.8*totals[i]):
        colors.append("indianred")
    else:
        colors.append("steelblue")
source = ColumnDataSource(data=dict(concepts=concepts, counts=counts, colors=colors, totals=totals, desc=description))

hover = HoverTool(
        tooltips=[
            ("Passed autograded tests", "@counts"),
            ("Total autograded tests", "@totals"),
            ("Description", "@desc"),
        ]
    )

div = Div(text="""<h1>\tModule: Entrance Test</h1>""", background="steelblue", width=800, height=50, style={'color': 'white', 'padding-left':'30px'})

show(div)
p = figure(y_range=concepts, x_range=(0,max_value), title="CT concepts", toolbar_location=None, tools=[hover])
p.hbar(y='concepts', height=0.5, left=0,
       right='counts', color='colors', source=source)

p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.xgrid.grid_line_color = None
# p.legend.location = "top_left"
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.xaxis.visible = False
p.title.text_font_size = "25px"
p.title.align = "center"

show(p)


# p1 = Paragraph(text="""Your text is initialized with the 'text' argument.  The
# remaining Paragraph arguments are 'width' and 'height'. For this example, those values
# are 200 and 100, respectively.""",
# width=200, height=100)
text_div = """<b><i>COMPUTATIONAL THINKING CONCEPTS</i></b></br>
              <b>Visualization</b> involves viewing your data using plots.</br>
              <b>Functionals</b> are blocks that your program is divided into.</br>
              <b>Sequences</b> are  a particular activity or task is expressed as a series of individual steps or instructions that can be executed by the computer.</br>
              <b>Loops</b> allow you to repeat a statement multiple times.</br>
              <b>Conditionals</b> allow you to make logical decisions based on a condition.</br>
              <b>Operators</b> provide support for mathematical, logical, and string expressions.</br>
              <b>Data</b> involves storing, retrieving, and updating values."""

div_concepts = Div(text=text_div, sizing_mode="stretch_both")
show(div_concepts)


# div = Div(text="""Your <a href="https://en.wikipedia.org/wiki/HTML">HTML</a>-supported text is initialized with the <b>text</b> argument.  The
# remaining div arguments are <b>width</b> and <b>height</b>. For this example, those values
# are <i>200</i> and <i>100</i>, respectively.""",
# width=200, height=100)

# show(div)

Loading BokehJS ...